# Create simulation settings file 
*This scripts creates a settings file that is needed as input to simulate time series*

*In order to simulate the time series, each of the time series components (seasonality, offset, remainder, and disturbance) are seperately simulated and subsequently summed. To ensure that the characteristics of these components are realistically simulated, the simulation is based on the characteristics of components of sampled time series (these can be characterised using eg the Char_opt_TS script).*

*The __seasonality__ is represented by an average sampled seasonal pattern having zero mean. Its amplitude can be adjusted and set within the range of the observed seasonal amplitudes. Additionally, drought events can be introduced by setting the seasonal cycle of a particular year to its minimum value. The __offset__ equals the average value of the observed time series. The __remainder__ represents the anomaly (in case no major disturbances are present). This component is defined by an ARMA model, fitted in the sampled remainder components, and its standard deviation can be adjusted. Finally, the __disturbance__ is represented by a step function with linear recovery. Here, the disturbance magnitude (magnitude of step function), disturbance timing (timing of step function) and recovery period (time needed to return to the predisturbance value) can be adjusted. After summing each of the components, missing values are introduced, with a user defined fraction.*

*In summary, the following characteristics can be adjusted:*
 - *seasonal amplitude*
 - *number of droughts*
 - *standard deviation of the remainder*
 - *disturbance magnitude*
 - *disturbance timing*
 - *recovery period*
 - *fraction of missing values*

*To be able to isolate the effect of a particular time series characteristic on the performance of a method, the following simulation set-up is used. First, a standard value for each of the characteristics is defined. Next, each  characteristic is allowed to seperately range over a set of values. For each value of the variable characteristic, n time series are simulated, while all other values of the time series characteristics are fixed to their standard value.*

*To ensure that the time series components and values of time series characteristics are realistic, the following data need to be derived from sampled time series:*

 - *Average seasonal pattern*
 - *Set of seasonal amplitudes from sampled time series*
 - *Offset*
 - *Standard deviation of remainder from sampled time series*
 - **
 - **
 - **



## 1. Import libraries

In [1]:
#library(gdata)

## 2. Inputs
*The script requires the following inputs:*

<font color=gray>    
 - __ifolder__: input folder, directory of the sampled time series characteristics files
 - __ofolder__: output folder, directory where the processed files will be stored
 - __nfolder__: notebook folder, directory where the jupyter notebook files are stored
 - __basename__: base name of the simulated time series files
 - __pol__: variable that is processed<font>

In [2]:
ifolder <- 'C:\\Users\\keers001\\OneDrive\ -\ WageningenUR\\RETURN\\Data\\RETURN\\20191019_SimulationOptSAR\\Landsat\\Char\\'  # folder with data characteristics
ofolder <- 'C:\\Users\\keers001\\OneDrive\ -\ WageningenUR\\RETURN\\Data\\RETURN\\20191019_SimulationOptSAR\\Landsat\\SimTS\\'  # folder where the simulated time series will be saved
nfolder <- 'C:\\Users\\keers001\\Dropbox\\output\\Jupyter_notebook\\'

basename <- c('LSTS_RndmSample_NoFire_5_Tree_80_scl_30_npnt_20000_VI')# base name of the data files
pol <- 'VI'#



## 3. Import data


In [3]:
system(paste0('jupyter-nbconvert.exe ', nfolder, 'sim_Functions.ipynb  --to script'))#, intern=FALSE
source(file.path(nfolder, 'sim_Functions.r'))

seasmax <- loadRData(file = file.path(ifolder, paste0(basename, '_seas', pol, 'max.rda')))# half seasonal amplitude
seasmean <- loadRData(file = file.path(ifolder, paste0(basename, '_seas', pol, 'mean.rda')))
trmean <- loadRData(file = file.path(ifolder, paste0(basename, '_Tr', pol, 'mean.rda')))
remsd <- loadRData(file = file.path(ifolder, paste0(basename, '_Rem_', pol, 'sd.rda')))
remcoef <- loadRData(file = file.path(ifolder, paste0(basename, '_Rem_', pol, 'coef.rda')))
missval <- loadRData(file = file.path(ifolder, paste0(basename, '_ts_', pol, 'MissVal.rda')))
  

[1] 0

## 4. Define simulation settings 
*Here, the simulation settings are defined:*

<font color=gray>    
 - __STnrep__: number of time series simulated per simulation run (one combination of all settings)
 - __STnobsYr__: number of observations simulated per year (this should equal the frequency of the sampled time series)
 - __DistMissVal__: defines how the introduced missing values should be distributed: random or at an equal interval
 - __STseasAv__: represents the standard seasonality<font>

In [4]:
  #-------------------------------------------------
  # standard settings and subsequently let each variable vary: 
STnrep <- 1000 
STnobsYr <- 365
DistMissVal <- 'random' # random or interval
STseasAv <- seasmean

STnDr <- 0 
STnyr <- 20
STdistMaglim <- -c(0.1,0.5) # magnitude randomly chosen between STdistMaglim dB
STdistT <- 10 # disturbance at random observation date in year STdistT
STdistReclim <- c(3,15)*STnobsYr # recovery time between STdistReclim years 
STqntl <- 0.5
STmissVal <- quantile(missval, STqntl)# if random - fraction of missing values, otherwise the frequency of observations
STseasAmp <- quantile(seasmax, STqntl)
STremSd <- quantile(remsd, STqntl)
STtrAv <- trmean

VnDr <- c(0,1) # number of droughts
Vnyr <- seq(6,40,3)# number of years
VdistMag <- -seq(0.1,0.5,0.1)# magnitude disturbance
VdistT <- seq(3,18,3)# timing disturbance
VdistRec <- seq(3,15,1)*STnobsYr# recovery disturbance
Vqntl <- c(.05, .275, .5, .725, .95)
VmissVal <- quantile(missval, Vqntl)
VseasAmp <- quantile(seasmax, Vqntl)# seasonal amplitude
VremSd <- quantile(remsd, Vqntl)# sd of remainder
  


## 5. Create settings list

In [5]:
STset <- list('var',
              STnrep,
              STnyr,
              STnobsYr,
              STnDr, 
              STseasAv, 
              STseasAmp,
              STtrAv,
              STremSd,
              STdistMaglim,
              STdistT, 
              STdistReclim, 
              STmissVal, 
              DistMissVal, 
              remcoef)
names(STset) <- c('variable',
                  'nrep',
                  'nyr',
                  'nobsYr',
                  'nDr', 
                  'seasAv', 
                  'seasAmp',
                  'trAv', 
                  'remSd', 
                  'distMag',
                  'distT', 
                  'distRec', 
                  'missVal', 
                  'DistMissVal',
                  'remcoef')


DRset <- STset
DRset$nDr <- VnDr
DRset$var <- c('nDr')
YRset <- STset
YRset$nyr <- Vnyr
YRset$var <- c('nyr')
SEASAMPset <- STset
SEASAMPset$seasAmp <- VseasAmp
SEASAMPset$var <- c('seasAmp')
REMSDset <- STset
REMSDset$remSd <- VremSd
REMSDset$var <- c('remSd')
DISTMAGset <- STset
DISTMAGset$distMag <- VdistMag
DISTMAGset$var <- c('distMag')
DISTTset <- STset
DISTTset$distT <- VdistT
DISTTset$var <- c('distT')
DISTRECset <- STset
DISTRECset$distRec <- VdistRec
DISTRECset$var <- c('distRec')
MISSVALset <- STset
MISSVALset$missVal <- VmissVal
MISSVALset$var <- c('missVal')


sttngs <- list(DRset, YRset, SEASAMPset, REMSDset, DISTMAGset, DISTTset, DISTRECset, MISSVALset)
names(sttngs) <- c('VnDr', 'Vnyr','VseasAmp','VremSd', 'VdistMag','VdistT', 'VdistRec', 'VmissVal')


## Export

In [6]:
save(sttngs, file = file.path(ofolder, paste0(basename, '_', pol, '_simTS_settings.rda')))
  